In [1]:
import os
import librosa
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
# Function to add noise to an audio sample (data augmentation)
def add_noise(audio, noise_factor=0.005):
    noise = np.random.randn(len(audio))
    augmented_audio = audio + noise_factor * noise
    return augmented_audio

In [3]:
# Load audio files and extract features
def extract_features(file_path, augment=False):
    try:
        audio, sample_rate = librosa.load(file_path, sr=None)
        if augment:
            audio = add_noise(audio)  # Apply noise augmentation

        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
        chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
        spec_contrast = librosa.feature.spectral_contrast(y=audio, sr=sample_rate)

        features = np.hstack([
            np.mean(mfccs.T, axis=0),
            np.mean(chroma.T, axis=0),
            np.mean(spec_contrast.T, axis=0)
        ])
        return features
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

In [4]:
# Function to process files concurrently using ThreadPoolExecutor
def process_files_concurrently(file_paths, label, augment=False):
    features_list = []
    labels_list = []

    with ThreadPoolExecutor() as executor:
        futures = {executor.submit(extract_features, file, augment): file for file in file_paths}
        for future in as_completed(futures):
            features = future.result()
            if features is not None:
                features_list.append(features)
                labels_list.append(label)

                if augment:
                    augmented_features = extract_features(futures[future], augment=True)
                    if augmented_features is not None:
                        features_list.append(augmented_features)
                        labels_list.append(label)

    return features_list, labels_list

In [5]:
# Function to process each folder separately
def process_folder(folder_name, folder_path, batch_size=50):
    print(f"Processing folder: {folder_name}")

    # Prepare paths for training, testing, and validation
    train_real_path = os.path.join(folder_path, 'training/real')
    train_fake_path = os.path.join(folder_path, 'training/fake')
    test_real_path = os.path.join(folder_path, 'testing/real')
    test_fake_path = os.path.join(folder_path, 'testing/fake')
    val_real_path = os.path.join(folder_path, 'validation/real')
    val_fake_path = os.path.join(folder_path, 'validation/fake')

    # Collect all the file paths
    train_real_files = [os.path.join(train_real_path, file) for file in os.listdir(train_real_path) if file.endswith('.wav')]
    train_fake_files = [os.path.join(train_fake_path, file) for file in os.listdir(train_fake_path) if file.endswith('.wav')]
    test_real_files = [os.path.join(test_real_path, file) for file in os.listdir(test_real_path) if file.endswith('.wav')]
    test_fake_files = [os.path.join(test_fake_path, file) for file in os.listdir(test_fake_path) if file.endswith('.wav')]
    val_real_files = [os.path.join(val_real_path, file) for file in os.listdir(val_real_path) if file.endswith('.wav')]
    val_fake_files = [os.path.join(val_fake_path, file) for file in os.listdir(val_fake_path) if file.endswith('.wav')]

    # Initialize lists for training, testing, and validation features and labels
    X_train, y_train, X_test, y_test, X_val, y_val = [], [], [], [], [], []

    # Process training files in batches
    for i in range(0, len(train_real_files), batch_size):
        batch_real = train_real_files[i:i+batch_size]
        real_train_features, real_train_labels = process_files_concurrently(batch_real, 0)
        X_train.extend(real_train_features)
        y_train.extend(real_train_labels)

        batch_fake = train_fake_files[i:i+batch_size]
        fake_train_features, fake_train_labels = process_files_concurrently(batch_fake, 1)
        X_train.extend(fake_train_features)
        y_train.extend(fake_train_labels)

    # Process testing files
    test_real_features, test_real_labels = process_files_concurrently(test_real_files, 0)
    X_test.extend(test_real_features)
    y_test.extend(test_real_labels)

    test_fake_features, test_fake_labels = process_files_concurrently(test_fake_files, 1)
    X_test.extend(test_fake_features)
    y_test.extend(test_fake_labels)

    # Process validation files
    val_real_features, val_real_labels = process_files_concurrently(val_real_files, 0)
    X_val.extend(val_real_features)
    y_val.extend(val_real_labels)

    val_fake_features, val_fake_labels = process_files_concurrently(val_fake_files, 1)
    X_val.extend(val_fake_features)
    y_val.extend(val_fake_labels)

    return X_train, y_train, X_test, y_test, X_val, y_val

In [6]:
# Define the paths for the different folders
base_path = '/kaggle/input/the-fake-or-real-dataset'
folders = {
    'for-2sec': os.path.join(base_path, 'for-2sec/for-2seconds'),
    'for-norm': os.path.join(base_path, 'for-norm/for-norm'),
    'for-original': os.path.join(base_path, 'for-original/for-original'),
    'for-rebec': os.path.join(base_path, 'for-rerec/for-rerecorded')
}



In [7]:
# Define the paths for the scenefake dataset
scenefake_path = '/kaggle/input/scenefake'
scenefake_folders = {
    'scenefake_train_real': os.path.join(scenefake_path, 'train/real'),
    'scenefake_train_fake': os.path.join(scenefake_path, 'train/fake'),
    'scenefake_test_real': os.path.join(scenefake_path, 'test/real'),
    'scenefake_test_fake': os.path.join(scenefake_path, 'test/fake'),
    'scenefake_val_real': os.path.join(scenefake_path, 'val/real'),
    'scenefake_val_fake': os.path.join(scenefake_path, 'val/fake')
}

In [8]:
# Define the scenefake folder paths
scenefake_folders = {
    'scenefake_train_real': '/kaggle/input/scenefake/train/real',
    'scenefake_train_fake': '/kaggle/input/scenefake/train/fake',
    'scenefake_test_real': '/kaggle/input/scenefake/eval/real',
    'scenefake_test_fake': '/kaggle/input/scenefake/eval/fake',
    'scenefake_val_real': '/kaggle/input/scenefake/dev/real',
    'scenefake_val_fake': '/kaggle/input/scenefake/dev/fake'
}


In [9]:
# Function to process the scenefake dataset
def process_scenefake_folder(scenefake_folders):
    X_train, y_train, X_test, y_test, X_val, y_val = [], [], [], [], [], []

    # Define the paths for training, testing, and validation
    train_real_path = scenefake_folders['scenefake_train_real']
    train_fake_path = scenefake_folders['scenefake_train_fake']
    test_real_path = scenefake_folders['scenefake_test_real']
    test_fake_path = scenefake_folders['scenefake_test_fake']
    val_real_path = scenefake_folders['scenefake_val_real']
    val_fake_path = scenefake_folders['scenefake_val_fake']

    # Collect all the file paths
    train_real_files = [os.path.join(train_real_path, file) for file in os.listdir(train_real_path) if file.endswith('.wav')]
    train_fake_files = [os.path.join(train_fake_path, file) for file in os.listdir(train_fake_path) if file.endswith('.wav')]
    test_real_files = [os.path.join(test_real_path, file) for file in os.listdir(test_real_path) if file.endswith('.wav')]
    test_fake_files = [os.path.join(test_fake_path, file) for file in os.listdir(test_fake_path) if file.endswith('.wav')]
    val_real_files = [os.path.join(val_real_path, file) for file in os.listdir(val_real_path) if file.endswith('.wav')]
    val_fake_files = [os.path.join(val_fake_path, file) for file in os.listdir(val_fake_path) if file.endswith('.wav')]

    # Process files and update lists
    real_train_features, real_train_labels = process_files_concurrently(train_real_files, 0)
    fake_train_features, fake_train_labels = process_files_concurrently(train_fake_files, 1)
    X_train.extend(real_train_features)
    y_train.extend(real_train_labels)
    X_train.extend(fake_train_features)
    y_train.extend(fake_train_labels)

    real_test_features, real_test_labels = process_files_concurrently(test_real_files, 0)
    fake_test_features, fake_test_labels = process_files_concurrently(test_fake_files, 1)
    X_test.extend(real_test_features)
    y_test.extend(real_test_labels)
    X_test.extend(fake_test_features)
    y_test.extend(fake_test_labels)

    real_val_features, real_val_labels = process_files_concurrently(val_real_files, 0)
    fake_val_features, fake_val_labels = process_files_concurrently(val_fake_files, 1)
    X_val.extend(real_val_features)
    y_val.extend(real_val_labels)
    X_val.extend(fake_val_features)
    y_val.extend(fake_val_labels)

    return X_train, y_train, X_test, y_test, X_val, y_val

# Initialize overall data lists for each step
overall_X_train, overall_y_train, overall_X_test, overall_y_test, overall_X_val, overall_y_val = [], [], [], [], [], []



In [10]:
# Process the scenefake dataset
print("Processing dataset from: scenefake")
X_train, y_train, X_test, y_test, X_val, y_val = process_scenefake_folder(scenefake_folders)
overall_X_train.extend(X_train)
overall_y_train.extend(y_train)
overall_X_test.extend(X_test)
overall_y_test.extend(y_test)
overall_X_val.extend(X_val)
overall_y_val.extend(y_val)



Processing dataset from: scenefake


In [11]:
# Process each folder in sequence
for folder_name, folder_path in folders.items():
    print(f"Processing dataset from: {folder_name}")
    
    X_train, y_train, X_test, y_test, X_val, y_val = process_folder(folder_name, folder_path, batch_size=50)
    
    # Update overall data lists
    overall_X_train.extend(X_train)
    overall_y_train.extend(y_train)
    overall_X_test.extend(X_test)
    overall_y_test.extend(y_test)
    overall_X_val.extend(X_val)
    overall_y_val.extend(y_val)



Processing dataset from: for-2sec
Processing folder: for-2sec


/usr/local/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


Processing dataset from: for-norm
Processing folder: for-norm


/usr/local/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1837
  warnings.warn(
/usr/local/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/usr/local/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1891
  warnings.warn(
/usr/local/lib/python3.10/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1690
  warnings.warn(


Processing dataset from: for-original
Processing folder: for-original
Processing dataset from: for-rebec
Processing folder: for-rebec


In [12]:
# Train the Random Forest model on the combined dataset
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(overall_X_train, overall_y_train)

# Make predictions on the test set
y_pred = model.predict(overall_X_test)



In [13]:
# Evaluate the model with additional metrics
accuracy = accuracy_score(overall_y_test, y_pred)
precision = precision_score(overall_y_test, y_pred)
recall = recall_score(overall_y_test, y_pred)
f1 = f1_score(overall_y_test, y_pred)

print(f"Overall Results:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}\n")

Overall Results:
Accuracy: 0.7721
Precision: 0.9167
Recall: 0.7571
F1-score: 0.8293



In [14]:
# Function to predict if a new audio file is fake or real
def predict_fake_audio(file_path):
    features = extract_features(file_path)
    if features is None:
        return "Error in processing audio file."

    prediction = model.predict([features])[0]
    if prediction == 0:
        return "Real audio"
    else:
        return "Fake audio"

In [15]:
import tensorflow as tf


E0000 00:00:1724447235.678923      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D0823 21:07:15.687866938      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0823 21:07:15.687887611      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0823 21:07:15.687891139      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0823 21:07:15.687893795      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0823 21:07:15.687896311      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Assuming the input shape is (13, 1, 1) as per the error
input_shape = (13, 1, 1)

# Create a simple CNN model with compatible kernel sizes
model = models.Sequential([
    layers.Conv2D(32, (1, 1), activation='relu', input_shape=input_shape),  # Adjust kernel size
    layers.MaxPooling2D((1, 1)),  # Adjust pooling size
    layers.Conv2D(64, (1, 1), activation='relu'),  # Adjust kernel size
    layers.MaxPooling2D((1, 1)),  # Adjust pooling size
    layers.Conv2D(64, (1, 1), activation='relu'),  # Adjust kernel size
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # Adjust output according to your classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()


/usr/local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1724447247.028121      13 service.cc:145] XLA service 0x5bcf4191d7f0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724447247.028179      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1724447247.028184      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1724447247.028187      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1724447247.028189      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1724447247.028192      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 0

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 13, 1, 32)      │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 1, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 1, 64)      │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 13, 1, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 13, 1, 64)      │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 832)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        53,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 60,298 (235.54 KB)

 Trainable params: 60,298 (235.54 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.save('/kaggle/working/DeepfakeAudio2.h5')


In [18]:
import joblib

# Save the RandomForest model
joblib.dump(model, 'deepfake_audio_rf_model.pkl')


['deepfake_audio_rf_model.pkl']

In [19]:
import joblib

# Load the RandomForest model
model = joblib.load('deepfake_audio_rf_model.pkl')


/usr/local/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [20]:
import joblib

# Define the path for saving the model
model_path = '/kaggle/working/deepfake_audio_rf_model.pkl'

# Save the RandomForest model
joblib.dump(model, model_path)


['/kaggle/working/deepfake_audio_rf_model.pkl']

In [21]:
# Save the model
model.save('/kaggle/working/DeepfakeDetection.h5')

print("Model saved as 'DeepfakeAudio.h5'")


Model saved as 'DeepfakeAudio.h5'


In [23]:
# Example usage:
new_audio_file = input("Enter the path to the audio file you want to check: ")

# Ensure that the file exists
if os.path.exists(new_audio_file):
    result = predict_fake_audio(new_audio_file)
    print(f"The prediction for the audio file is: {result}")
else:
    print("The specified file does not exist. Please check the path and try again.")


Enter the path to the audio file you want to check:  /kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training/fake/file10007.mp3.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32,), dtype=float32). Expected shape (None, 13, 1, 1), but input has incompatible shape (32,)[0m

Arguments received by Sequential.call():
  • inputs=('tf.Tensor(shape=(32,), dtype=float32)',)
  • training=False
  • mask=('None',)